In [3]:
import fonction as f
import numpy as np
import os 

In [4]:
masse_linéique = [3.61 * 10**(-3), 50 * 10**(-3), 80 * 10 **(-3)] #masse linéique de corde
tension = np.linspace(50,100,10) 
#faire varier B et eta?